# 미국 주식 데이터 수집하기

퀀트 투자의 장점은 데이터만 있다면 동일한 투자 전략을 전세계 모든 국가에 적용할 수 있다는 점이다. 이번 장에서는 미국 전 종목의 티커 수집 및 주가, 재무제표, 가치지표를 다운로드 해보도록 하겠다.

## 티커 수집하기

우리나라는 거래소가 '한국거래소' 한 곳 뿐이지만, 미국은 매우 많은 거래소가 존재한다. 따라서 동일한 종목이 여러 거래소에서 거래되는 일도 발생한다. 그 중에서도 가장 규모가 큰 거래소는 뉴욕거래소(NYSE), 나스닥(NASDAQ), 아멕스(AMEX)이며, 해당 거래소에 상장된 종목들은 나스닥 홈페이지에서 제공된다.

```
https://www.nasdaq.com/market-activity/stocks/screener
```

위 사이트에 접속하여 개발자 도구 화면을 연 후 하단의 [Download CSV] 버튼을 누르면, 해당 데이터를 받아오는 웹주소가 표시된다.

```{figure} image/data_us/us_ticker.png
---
name: us_ticker
---
미국의 상장종목 리스트
```

```
https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=25&offset=0&download=true
```

해당 주소에 접속하면, 각 종목에 대한 정보가 JSON 형태로 제공된다. 이를 크롤링 해보도록 하겠다.

In [1]:
import pandas as pd
import requests as rq

url = 'https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=25&offset=0&download=true'
headers = {
    "User-Agent":
    "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"
}

data = rq.get(url, headers=headers)
data_json = data.json()['data']
us_ticker = pd.DataFrame(data_json['rows'])

us_ticker.head()

,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
0,A,Agilent Technologies Inc. Common Stock,$136.87,6.56,5.034%,2077717,39326538330.00,United States,1999,Biotechnology: Laboratory Analytical Instruments,Industrials,/market-activity/stocks/a
1,AA,Alcoa Corporation Common Stock,$40.57,0.48,1.197%,3863800,10480859480.00,United States,2016,Aluminum,Industrials,/market-activity/stocks/aa
2,AACG,ATA Creativity Global American Depositary Shares,$1.01,0.0776,8.323%,218765,32320131.00,China,2008,Other Consumer Services,Real Estate,/market-activity/stocks/aacg
3,AACT,Ares Acquisition Corporation II Class A Ordina...,$10.875,-0.005,-0.046%,55446,0.00,,2023,Blank Checks,Finance,/market-activity/stocks/aact
4,AADI,Aadi Bioscience Inc. Common Stock,$1.99,0.00,0.00%,25856,48983520.00,United States,,Biotechnology: Pharmaceutical Preparations,Health Care,/market-activity/stocks/aadi


1. 먼저 url과 헤더정보를 입력하며, 헤더의 User-Agent에는 웹브라우저 구별을 입력해준다. 해당 사이트는 크롤러와 같이 정체가 불분명한 웹브라우저를 통한 접속이 막혀 있어, 마치 모질라 혹은 크롬을 통해 접속한 것 처럼 데이터를 요청한다. 다양한 웹브라우저 리스트는 다음 사이트에서 확인할 수 있다.
http://www.useragentstring.com/pages/useragentstring.php
2. `get()` 함수를 통해 페이지의 데이터를 불러온다.
3. `json()` 함수를 통해 json 형태로 불러온 후, data 부분을 선택한다.
4. json 형태를 데이터프레임 형태로 변경한다.

## 티커 클렌징

먼저 위에서 받은 데이터에서 종목 수를 확인해보도록 하자.

In [2]:
len(us_ticker)

7008

종목수를 살펴보면 상당히 많은 종목이 존재한다. 그러나 미국의 경우 동일한 종목이 여러 거래소에 상장된 경우가 있으므로, 이를 먼저 제거해주도록 한다. symbol과 name열 중 중복되는 종목이 있으면 첫번쨰 데이터만을 남기도록 한다.

In [3]:
us_ticker = us_ticker.loc[~us_ticker.symbol.duplicated(keep='first'), :]
us_ticker = us_ticker.loc[~us_ticker.name.duplicated(keep='first'), :]

len(us_ticker)

6999

다음으로 위에서 받은 데이터 중 모든 종목이 투자에 적합한 종목이 아니므로 클렌징 처리를 해주도록 한다.

In [4]:
us_ticker['marketCap'] = pd.to_numeric(us_ticker['marketCap'])

 먼저 시가총액에 해당하는 열을 숫자 형태로 바꿔준다. 이제 어떠한 종목이 일반적인 종목이 아닌지 탐색해보도록 하자.

In [5]:
import numpy as np

display(us_ticker.loc[us_ticker['marketCap'] == 0].head(),
        us_ticker[np.isnan(us_ticker['marketCap'])].head(),
        us_ticker.loc[us_ticker['name'].str.contains('%')].head())

,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
3,AACT,Ares Acquisition Corporation II Class A Ordina...,$10.875,-0.005,-0.046%,55446,0.0,,2023,Blank Checks,Finance,/market-activity/stocks/aact
6,AAM,AA Mission Acquisition Corp. Class A Ordinary ...,$10.05,0.01,0.10%,65394,0.0,,2024,,,/market-activity/stocks/aam
43,ACAB,Atlantic Coastal Acquisition Corp. II Class A ...,$11.55,0.20,1.762%,17487,0.0,United States,2022,Blank Checks,Finance,/market-activity/stocks/acab
44,ACABU,Atlantic Coastal Acquisition Corp. II Unit,$12.54,0.00,0.00%,5,0.0,United States,2022,Blank Checks,Finance,/market-activity/stocks/acabu
72,ACP,abrdn Income Credit Strategies Fund Common Shares,$6.44,-0.05,-0.77%,714776,0.0,United States,2011,Investment Managers,Finance,/market-activity/stocks/acp


,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
30,ABR^D,Arbor Realty Trust 6.375% Series D Cumulative ...,$19.00,-0.79,-3.992%,14644,NaN,United States,,,,/market-activity/stocks/abr^d
31,ABR^E,Arbor Realty Trust 6.25% Series E Cumulative R...,$19.4996,-0.0504,-0.258%,2561,NaN,United States,,,,/market-activity/stocks/abr^e
32,ABR^F,Arbor Realty Trust 6.25% Series F Fixed-to-Flo...,$20.52,-0.17,-0.822%,20739,NaN,United States,,,,/market-activity/stocks/abr^f
73,ACP^A,abrdn Income Credit Strategies Fund 5.250% Ser...,$23.4732,-0.6669,-2.763%,421,NaN,United States,,,,/market-activity/stocks/acp^a
75,ACR^C,ACRES Commercial Realty Corp. 8.625% Fixed-to-...,$24.89,0.02,0.08%,15288,NaN,United States,,,,/market-activity/stocks/acr^c


,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
22,ABLLL,Abacus Life Inc. 9.875% Fixed Rate Senior Note...,$26.40,0.00,0.00%,227,1.970796e+09,United States,,Investment Managers,Finance,/market-activity/stocks/ablll
30,ABR^D,Arbor Realty Trust 6.375% Series D Cumulative ...,$19.00,-0.79,-3.992%,14644,NaN,United States,,,,/market-activity/stocks/abr^d
31,ABR^E,Arbor Realty Trust 6.25% Series E Cumulative R...,$19.4996,-0.0504,-0.258%,2561,NaN,United States,,,,/market-activity/stocks/abr^e
32,ABR^F,Arbor Realty Trust 6.25% Series F Fixed-to-Flo...,$20.52,-0.17,-0.822%,20739,NaN,United States,,,,/market-activity/stocks/abr^f
53,ACGLN,Arch Capital Group Ltd. Depositary Shares each...,$20.255,-0.005,-0.025%,60938,7.617052e+09,Bermuda,,Property-Casualty Insurers,Finance,/market-activity/stocks/acgln


첫째, 시가총액이 0인 종목, 시가총액이 NaN인 종목, 종목 이름에 '%'가 들어간 종목을 살펴보면 Acquisition 혹은 Warrant 등이거나 전환사채, 우선주 등 보통주가 아니다. 따라서 제외하는 것이 좋다.

In [6]:
us_ticker.loc[us_ticker['name'].str.contains('ETF | Fund')].head()

,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
72,ACP,abrdn Income Credit Strategies Fund Common Shares,$6.44,-0.05,-0.77%,714776,0.0,United States,2011,Investment Managers,Finance,/market-activity/stocks/acp
73,ACP^A,abrdn Income Credit Strategies Fund 5.250% Ser...,$23.4732,-0.6669,-2.763%,421,NaN,United States,,,,/market-activity/stocks/acp^a
84,ACV,Virtus Diversified Income & Convertible Fund C...,$20.645,0.015,0.073%,41507,213943185.0,United States,2015,Finance/Investors Services,Finance,/market-activity/stocks/acv
115,ADX,Adams Diversified Equity Fund Inc.,$21.73,0.07,0.323%,223890,0.0,United States,,Investment Managers,Finance,/market-activity/stocks/adx
120,AEF,abrdn Emerging Markets Equity Income Fund Inc....,$5.36,-0.03,-0.557%,36797,575463000.0,United States,,Finance/Investors Services,Finance,/market-activity/stocks/aef


둘째, 종목명에 'ETF' 혹은 'Fund'가 들어간 종목은 펀드에 해당하므로 제외한다.

In [7]:
us_ticker.loc[us_ticker['name'].str.contains(
    'Depositary | each representing')].head()

,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
2,AACG,ATA Creativity Global American Depositary Shares,$1.01,0.0776,8.323%,218765,3.232013e+07,China,2008,Other Consumer Services,Real Estate,/market-activity/stocks/aacg
19,ABEV,Ambev S.A. American Depositary Shares (Each re...,$2.15,-0.03,-1.376%,19143857,3.387074e+10,Brazil,2013,Beverages (Production/Distribution),Consumer Staples,/market-activity/stocks/abev
40,ABVX,Abivax SA American Depositary Shares,$10.27,-0.05,-0.484%,259218,6.461625e+08,France,2023,Biotechnology: Pharmaceutical Preparations,Health Care,/market-activity/stocks/abvx
53,ACGLN,Arch Capital Group Ltd. Depositary Shares each...,$20.255,-0.005,-0.025%,60938,7.617052e+09,Bermuda,,Property-Casualty Insurers,Finance,/market-activity/stocks/acgln
54,ACGLO,Arch Capital Group Ltd. Depositary Shares Each...,$23.40,-0.04,-0.171%,25400,8.799753e+09,Bermuda,,Property-Casualty Insurers,Finance,/market-activity/stocks/acglo


마지막으로 종목명에 'Depositary'나 'each representing'가 들어간 종목은 ADR 이므로 제외한다.

In [8]:
us_ticker = us_ticker.loc[~(us_ticker['marketCap'] == 0)]
us_ticker = us_ticker.loc[~(np.isnan(us_ticker['marketCap']))]
us_ticker = us_ticker.loc[~(us_ticker['name'].str.contains('%'))]
us_ticker = us_ticker.loc[~us_ticker['name'].str.contains('ETF | Fund')]
us_ticker = us_ticker.loc[~us_ticker['name'].str.contains('Depositary')]
us_ticker = us_ticker.loc[~us_ticker['name'].str.contains('each representing')]

len(us_ticker)

5417

이전에 비해 종목수가 현격하게 줄어들었다. 물론 해당 과정을 거쳐도 다중 클래스가 상장되어 있는 등 클렌징 처리를 할 종목이 많지만 그 수가 매우 적으므로, 효율성 측면에서 위의 과정 정도만으로도 충분하다. 이 중 필요한 열 선택 및 기준일(오늘 날짜)을 입력해준다.

In [9]:
from datetime import date, timedelta

date.today()

us_ticker = us_ticker[[
    'symbol', 'name', 'marketCap', 'country', 'ipoyear', 'industry', 'sector'
]]
us_ticker['date'] = date.today().strftime("%Y%m%d")
us_ticker['date'] = pd.to_datetime(us_ticker['date'])
us_ticker = us_ticker.replace({np.nan: None})

us_ticker.head(10)

,symbol,name,marketCap,country,ipoyear,industry,sector,date
0,A,Agilent Technologies Inc. Common Stock,3.932654e+10,United States,1999,Biotechnology: Laboratory Analytical Instruments,Industrials,2024-11-03
1,AA,Alcoa Corporation Common Stock,1.048086e+10,United States,2016,Aluminum,Industrials,2024-11-03
4,AADI,Aadi Bioscience Inc. Common Stock,4.898352e+07,United States,,Biotechnology: Pharmaceutical Preparations,Health Care,2024-11-03
5,AAL,American Airlines Group Inc. Common Stock,8.897554e+09,United States,,Air Freight/Delivery Services,Consumer Discretionary,2024-11-03
7,AAME,Atlantic American Corporation Common Stock,3.325161e+07,United States,,Life Insurance,Finance,2024-11-03
8,AAOI,Applied Optoelectronics Inc. Common Stock,6.886838e+08,United States,2013,Semiconductors,Technology,2024-11-03
9,AAON,AAON Inc. Common Stock,9.192562e+09,United States,,Industrial Machinery/Components,Industrials,2024-11-03
10,AAP,Advance Auto Parts Inc.,2.163193e+09,United States,,Auto & Home Supply Stores,Consumer Discretionary,2024-11-03
11,AAPL,Apple Inc. Common Stock,3.389154e+12,United States,1980,Computer Manufacturing,Technology,2024-11-03
12,AAT,American Assets Trust Inc. Common Stock,1.622418e+09,United States,2011,Real Estate Investment Trusts,Real Estate,2024-11-03


이제 위의 데이터를 DB에 저장한다. 먼저 SQL에서 다음의 쿼리를 통해 테이블(us_ticker)을 만든다.

In [ ]:
use stock_db;

create table us_ticker
(
    symbol varchar(20) not null,
    name varchar(300),
    marketCap float,
    country varchar(20),
    ipoyear varchar(4),
    industry varchar(100),
    sector varchar(30),
    date date,
    primary key(symbol, date)
);

마지막으로 티커 데이터를 해당 테이블에 저장한다.

In [11]:
import pymysql

con = pymysql.connect(user='root',
                      passwd='1346',
                      host='127.0.0.1',
                      db='stock_db',
                      charset='utf8')

mycursor = con.cursor()
query = """
    insert into us_ticker2 (symbol, name, marketCap, country, ipoyear, industry, sector, date)
    values (%s,%s,%s,%s,%s,%s,%s,%s) as new
    on duplicate key update
    name=new.name,marketCap=new.marketCap,country=new.country,
    ipoyear=new.ipoyear,industry=new.industry,sector=new.sector;    
"""

args = us_ticker.values.tolist()

mycursor.executemany(query, args)
con.commit()

con.close()

## 주가 다운로드

미국 뿐만 아니라 전 세계 주가(한국 포함)의 경우 야후 파이낸스에서 제공하고 있다.

```
https://finance.yahoo.com/
```

사이트에서 종목 티커를 검색한 후 [Historical Data] 탭을 선택하면 확인 및 다운로드가 가능하다. 또한 pandas_datareader 패키지의 `DataReader()` 함수를 사용하면 야후 API를 통해 주가 데이터를 매우 손쉽게 다운로드 받을 수도 있다. 예시로써 애플(AAPL)의 주가를 다운로드 받아보도록 하자.

```{figure} image/data_us/yahoo_price.png
---
name: yahoo_price
---
야후에서 제공하는 주가 데이터
```

In [15]:
import yfinance as yf

price = yf.download('AAPL',progress=False,start='2000-01-01')
price.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2000-01-03 00:00:00+00:00,0.844004,0.999442,1.004464,0.907924,0.936384,535796800
2000-01-04 00:00:00+00:00,0.772846,0.915179,0.987723,0.903460,0.966518,512377600
2000-01-05 00:00:00+00:00,0.784155,0.928571,0.987165,0.919643,0.926339,778321600
2000-01-06 00:00:00+00:00,0.716295,0.848214,0.955357,0.848214,0.947545,767972800
2000-01-07 00:00:00+00:00,0.750226,0.888393,0.901786,0.852679,0.861607,460734400


`DataReader()` 함수 내에 티커와 출처에 해당하는 'yahoo'를 입력하면 주가 정보를 매우 손쉽게 받을 수 있다.

```{note}
국내 주가 역시 야후 파이낸스를 통해 다운로드 받을 수 있다. 그러나 일부 중소형주의 경우 데이터가 존재하지 않는 문제가 있어 국내 사이트를 이용해 수집하는 것을 권장한다.
```

### 전 종목 주가 다운로드

미국 데이터 역시 국내 전종목 주가를 다운로드 받고 DB에 저장했던것과 동일하게 `for loop` 구문을 이용하면 된다. 먼저 SQL에서 미국 주가 데이터에 해당하는 테이블(us_price)를 만든다.

In [ ]:
use stock_db;

create table us_price
(
    Date date,
    High double,
    Low double,
    Open double,
    Close double,
    Volume double,
    Adj_Close double,
    ticker varchar(20),
    primary key(Date, ticker)
);

파이썬에서 아래 코드를 실행하면 for loop 구문을 통해 전종목 주가가 DB에 저장된다.

In [17]:
# 패키지 불러오기

import pymysql
import pandas as pd
import yfinance as yf
from datetime import date
import time
from tqdm import tqdm

# DB 연결
con = pymysql.connect(user='root',
                      passwd='1346',
                      host='127.0.0.1',
                      db='stock_db',
                      charset='utf8')

mycursor = con.cursor()

# 티커리스트 불러오기
ticker_list = pd.read_sql("""
select * from us_ticker2
where date = (select max(date) from us_ticker2);
""",
                          con=con)

# DB 저장 쿼리
query = """
    insert into us_price (Date, High, Low, Open, Close, Volume, Adj_Close, ticker)
    values (%s, %s,%s,%s,%s,%s,%s,%s) as new
    on duplicate key update
    High = new.High, Open = new.Open, Close = new.Close,
    Volume = new.Volume, Adj_Close = new.Adj_Close;
"""

# 오류 발생시 저장할 데이터프레임 생성
error_list = []
price_empty = pd.DataFrame({
    'Date': date.today().strftime("%Y-%m-%d"),
    'High': [None],
    'Low': [None],
    'Open': [None],
    'Close': [None],
    'Volumne': [None],
    'Adj_Close': [None]
})

# 전종목 주가 다운로드 및 저장
for i in tqdm(range(0, len(ticker_list))):

    # 빈 데이터프레임 복사
    price = price_empty.copy()

    # 티커 선택
    ticker = ticker_list['symbol'][i]

    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:

        # url 생성
        price = yf.download(ticker,progress=False)

        # 데이터 클렌징
        price = price.reset_index()
        price['ticker'] = ticker
        price.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)

    except:

        # 오류 발생시 빈 데이터프레임을 불러온 후 다음 루프로 이동
        price['ticker'] = ticker
        error_list.append(ticker)

    # 주가 데이터를 DB에 저장
    args = price.values.tolist()

    mycursor.executemany(query, args)
    con.commit()

    # 타임슬립 적용
    time.sleep(2)

# DB 연결 종료
con.close()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13404\103438454.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ticker_list = pd.read_sql("""
  0%|          | 18/5417 [01:27<6:06:13,  4.07s/it]
1 Failed download:
['ABLLW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
  0%|          | 20/5417 [01:32<4:51:42,  3.24s/it]
1 Failed download:
['ABLVW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
  1%|          | 31/5417 [02:13<5:13:14,  3.49s/it]
1 Failed download:
['ABVEW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
  1%|          | 57/5417 [03:54<6:30:51,  4.38s/it]
1 Failed download:
['ACONW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
  1%|▏      

In [18]:
error_list

[]

1. DB에 연결한다.
2. 기준일이 최대, 즉 최근일 기준 보통주에 해당하는 종목 리스트(ticker_list)만 불러온다.
3. DB에 저장할 쿼리(query)를 입력한다.
4. 페이지 오류, 통신 오류 등 오류가 발생할 경우 저장할 빈 데이터프레임(price_empty)을 생성한다.
5. `for loop` 구문을 통해 전종목 주가를 다운로드 받으며, 진행상황을 알기위해 `tqdm()` 함수를 이용한다.
6. `DataReader()` 함수를 통해 야후 파이낸스에서 주가를 받은 후 클렌징 처리한다.
7.  `try except` 문을 통해 오류가 발생시 위에서 생성한 빈 데이터프레임을 이용한다.
8. 주가 데이터를 DB에 저장한다.
9. 무한 크롤링을 방지하기 위해 한 번의 루프가 끝날 때마다 타임슬립을 적용한다.
10. 모든 작업이 끝나면 DB와의 연결을 종료한다.

```{figure} image/data_us/sql_price.png
---
name: sql_price
---
미국 주가 테이블
```

## 재무제표 크롤링

재무제표 역시 야후 파이낸스에서 구할 수 있다. [Financials] 탭을 클릭하면 연간 및 분기 기준 재무제표를 제공하고 있다. 해당 데이터를 다운로드 받을 수 있는 여러 패키지가 존재하며, 본 책에서는 그 중에서도 `yahoo_fin` 패키지를 사용하도록 하겠다. 해당 패키지의 자세한 설명은 아래 사이트에서 확인할 수 있다.

```
http://theautomatic.net/yahoo_fin-documentation/
```

```{figure} image/data_us/yahoo_fs.png
---
name: yahoo_fs
---
야후에서 제공하는 재무제표 데이터
```

해당 패키지의 `get_financials()` 함수를 이용하면 손익계산서, 재무상태표, 현금흐름표를 한번에 다운로드 받을 수 있으며, 예시로 애플(AAPL) 종목의 연간 재무제표를 받아보도록 하겠다.

In [12]:
from yahooquery import Ticker

data = Ticker('AAPL')
data.asset_profile
data.summary_detail
data.history()

c:\Users\kjh\anaconda3\lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
c:\Users\kjh\anaconda3\lib\site-packages\yahooquery\ticker.py:1333: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["dividends"].fillna(0, inplace=True)


open        high         low       close    volume  \
symbol date                                                                   
AAPL   2024-01-02  187.149994  188.440002  183.889999  185.639999  82488700   
       2024-01-03  184.220001  185.880005  183.429993  184.250000  58414500   
       2024-01-04  182.149994  183.089996  180.880005  181.910004  71983600   
       2024-01-05  181.990005  182.759995  180.169998  181.179993  62303300   
       2024-01-08  182.089996  185.600006  181.500000  185.559998  59144500   
...                       ...         ...         ...         ...       ...   
       2024-10-28  233.320007  234.729996  232.550003  233.399994  36087100   
       2024-10-29  233.100006  234.330002  232.320007  233.669998  35417200   
       2024-10-30  232.610001  233.470001  229.550003  230.100006  47070900   
       2024-10-31  229.339996  229.830002  225.369995  225.910004  64370100   
       2024-11-01  220.970001  225.350006  220.270004  222.910004  65242200   

                     adjclose  dividends  
symbol date                               
AAPL   2024-01-02  184.938217        0.0  
       2024-01-03  183.553467        0.0  
       2024-01-04  181.222336        0.0  
       2024-01-05  180.495071        0.0  
       2024-01-08  184.858521        0.0  
...                       ...        ...  
       2024-10-28  233.399994        0.0  
       2024-10-29  233.669998        0.0  
       2024-10-30  230.100006        0.0  
       2024-10-31  225.910004        0.0  
       2024-11-01  222.910004        0.0  

[212 rows x 7 columns]

In [13]:
data_y = data.all_financial_data(frequency='a')
data_y

,asOfDate,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccumulatedDepreciation,AvailableForSaleSecurities,BasicAverageShares,BasicEPS,BeginningCashPosition,...,TotalExpenses,TotalLiabilitiesNetMinorityInterest,TotalNonCurrentAssets,TotalNonCurrentLiabilitiesNetMinorityInterest,TotalOperatingIncomeAsReported,TotalRevenue,TotalTaxPayable,TradeandOtherPayablesNonCurrent,TreasurySharesNumber,WorkingCapital
symbol,,,,,,,,,,,,,,,,,,,,,
AAPL,2020-09-30,12M,USD,4.229600e+10,1.612000e+10,-6.676000e+10,1.008870e+11,1.735212e+10,3.31,5.022400e+10,...,2.082270e+11,2.585490e+11,1.801750e+11,1.531570e+11,6.628800e+10,2.745150e+11,NaN,2.817000e+10,NaN,3.832100e+10
AAPL,2021-09-30,12M,USD,5.476300e+10,2.627800e+10,-7.028300e+10,1.278770e+11,1.670127e+10,5.67,3.978900e+10,...,2.568680e+11,2.879120e+11,2.161660e+11,1.624310e+11,1.089490e+11,3.658170e+11,NaN,2.468900e+10,NaN,9.355000e+09
AAPL,2022-09-30,12M,USD,6.411500e+10,2.818400e+10,-7.234000e+10,1.208050e+11,1.621596e+10,6.15,3.592900e+10,...,2.748910e+11,3.020830e+11,2.173500e+11,1.481010e+11,1.194370e+11,3.943280e+11,6.552000e+09,1.665700e+10,NaN,-1.857700e+10
AAPL,2023-09-30,12M,USD,6.261100e+10,2.950800e+10,-7.088400e+10,1.005440e+11,1.574423e+10,6.16,2.497700e+10,...,2.689840e+11,2.904370e+11,2.090170e+11,1.451290e+11,1.143010e+11,3.832850e+11,8.819000e+09,1.545700e+10,0.0,-1.742000e+09


In [14]:
import numpy as np

data_y.reset_index(inplace=True)
data_y = data_y.loc[:,~data_y.columns.isin(['periodType','currencyCode'])]
data_y = data_y.melt(id_vars = ['symbol','asOfDate'])
data_y = data_y.replace([np.nan],None)
data_y['freq'] = 'y'
data_y.columns = ['ticker','date','account','value','freq']
data_y

,ticker,date,account,value,freq
0,AAPL,2020-09-30,AccountsPayable,42296000000.0,y
1,AAPL,2021-09-30,AccountsPayable,54763000000.0,y
2,AAPL,2022-09-30,AccountsPayable,64115000000.0,y
3,AAPL,2023-09-30,AccountsPayable,62611000000.0,y
4,AAPL,2020-09-30,AccountsReceivable,16120000000.0,y
...,...,...,...,...,...
655,AAPL,2023-09-30,TreasurySharesNumber,0.0,y
656,AAPL,2020-09-30,WorkingCapital,38321000000.0,y
657,AAPL,2021-09-30,WorkingCapital,9355000000.0,y
658,AAPL,2022-09-30,WorkingCapital,-18577000000.0,y


In [25]:
import yahoo_fin.stock_info as si

data_y = si.get_financials('AAPL', yearly=True, quarterly=False)
data_y.keys()

ModuleNotFoundError: No module named 'yahoo_fin'

인자에 `yearly = True, quarterly = False`는 연간 재무제표를 의미하며, 딕셔너리 형태로 세 종류의 재무제표가 다운로드 되었다. 이를 하나의 데이터프레임으로 바꿔주도록 한다.

In [ ]:
import pandas as pd

data_fs_y = pd.concat([v for k, v in data_y.items()])
data_fs_y = data_fs_y.stack().reset_index()
data_fs_y.columns = ['account', 'date', 'value']
data_fs_y['freq'] = 'y'

data_fs_y.head()

1. 딕셔너리의 value에 해당하는 부분만을 선택한 후 `concat()` 함수를 통해 데이터프레임 형태로 합쳐준다.
2. `stack()` 함수를 통해 데이터를 위에서 아래로 길게 재구조화 해주며, `reset_index()` 함수를 통해 인덱스를 초기화한다.
3. 열 이름을 변경한다.
4. freq 열에 연간에 해당하는 'y'를 입력한다.

이처럼 패키지를 이용하여 미국 데이터도 매우 쉽게 다운로드 받을 수 있다. 분기별 재무제표를 받는법도 위와 같으며, 인자만 `yearly=False, quarterly=True`로 변경하면 된다.

In [ ]:
data_q = si.get_financials('AAPL', yearly=False, quarterly=True)
data_fs_q = pd.concat([v for k, v in data_q.items()])
data_fs_q = data_fs_q.stack().reset_index()
data_fs_q.columns = ['account', 'date', 'value']
data_fs_q['freq'] = 'q'

data_fs_q.head()

### 전 종목 재무제표 다운로드

`for loop` 구문을 이용하여 전 종목 재무제표를 다운로드 받도록 하겠다. 먼저 SQL에서 미국 재무제표 데이터에 해당하는 테이블(us_fs)를 만든다.

In [ ]:
use stock_db;

create table us_fs
(
    ticker varchar(20),
    date date,
    account varchar(100),
    value double,
    freq varchar(1),
        
    primary key(account, date, ticker, freq)
);

이제 파이썬에서 아래 코드를 실행하면 for loop 구문을 통해 전 종목 재무제표가 DB에 저장된다.

In [15]:
# 패키지 불러오기
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from yahooquery import Ticker
import numpy as np
import time
from tqdm import tqdm

# DB 연결
con = pymysql.connect(
    user='root', passwd='1346', host='127.0.0.1', db='stock_db', charset='utf8'
)

mycursor = con.cursor()

# 티커리스트 불러오기
ticker_list = pd.read_sql("""
select * from us_ticker2
where date = (select max(date) from us_ticker2);
""", con = con)


# DB 저장 쿼리
query_fs = """
    insert into us_fs2 (ticker,date,account,value,freq)
    values (%s,%s,%s,%s,%s) as new
    on duplicate key update
    value = new.value;
"""


# 오류 발생시 저장할 데이터프레임 생성
error_list = []



# 전종목 주가 다운로드 및 저장

for i in tqdm(range(0, len(ticker_list))): 

    
    
    # 티커 선택
    ticker = ticker_list['symbol'][i]        
    
    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:
        
       data = Ticker(ticker)

       # 재무제표 다운로드
       
       # 연간
       data_y = data.all_financial_data(frequency='a')
       data_y.reset_index(inplace=True)
       data_y = data_y.loc[:,~data_y.columns.isin(['periodType','currencyCode'])]
       data_y = data_y.melt(id_vars = ['symbol','asOfDate'])
       data_y = data_y.replace([np.nan],None)
       data_y['freq'] = 'y'
       data_y.columns = ['ticker','date','account','value','freq']

       # 분기
       data_q = data.all_financial_data(frequency='q')
       data_q.reset_index(inplace=True)
       data_q = data_q.loc[:,~data_q.columns.isin(['periodType','currencyCode'])]
       data_q = data_q.melt(id_vars = ['symbol','asOfDate'])
       data_q = data_q.replace([np.nan],None)
       data_q['freq'] = 'q'
       data_q.columns = ['ticker','date','account','value','freq']
       
       # 합치기
       data_fs = pd.concat([data_y, data_q], axis = 0)
                   

       # 주가 데이터를 DB에 저장
       args  = data_fs.values.tolist() 
       mycursor.executemany(query_fs, args)
       con.commit()

    except:
        
        # 오류 발생시 빈 데이터프레임을 불러온 후 다음 루프로 이동
        print(ticker)
        error_list.append(ticker)   
       
    

    
    # 타임슬립 적용
    time.sleep(2)    
    
# DB 연결 종료

con.close()    

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14524\3106205753.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ticker_list = pd.read_sql("""
  5%|▌         | 273/5417 [31:16<9:17:52,  6.51s/it] 

AMTM


 10%|▉         | 534/5417 [1:04:10<11:01:49,  8.13s/it]

BANFP


 11%|█         | 590/5417 [1:11:46<10:42:01,  7.98s/it]

...

ZJK


100%|█████████▉| 5404/5417 [10:38:15<01:14,  5.75s/it]

ZOOZW


100%|██████████| 5417/5417 [10:39:32<00:00,  7.08s/it]


NameError: name 'engine' is not defined

In [16]:
con.close()

1. DB에 연결한다.
2. 기준일이 최대, 즉 최근일 기준 보통주에 해당하는 종목 리스트(ticker_list)만 불러온다.
3. DB에 저장할 쿼리(query)를 입력한다.
4. 페이지 오류, 통신 오류 등 오류가 발생할 경우 저장할 빈 데이터프레임(fs_empty)을 생성한다.
5. `for loop` 구문을 통해 전종목 재무제표를 다운로드 받으며, 진행상황을 알기위해 `tqdm()` 함수를 이용한다.
6. `get_financials()` 함수를 이용해 연간 및 분기 재무제표를 받은 후, 두 테이블을 `concat()` 함수를 통해 행으로 묶어준다.
7. 재무제표 데이터를 DB에 저장한다.
8. 무한 크롤링을 방지하기 위해 한 번의 루프가 끝날 때마다 타임슬립을 적용한다.
9. 모든 작업이 끝나면 DB와의 연결을 종료한다.

```{figure} image/data_us/sql_fs.png
---
name: sql_fs
---
미국 재무제표 테이블
```

```{note}
미국 종목들의 가치지표는 국내 재무제표 데이터를 이용해 가치지표를 계산했던 것과 동일한 방법으로 계산할 수 있으므로, 이는 생략하도록 한다.
```